# How to use `cugraph-nx` to accelerate graph analysis while using networkX 

Using the cugraph plugin for NetworkX requires Python 3.9, 3.10, or 3.11, in addition to the latest development version of NetworkX. Follow the instructions from `INSTALL.rst` on the NetworkX repository for instructions on installing the development version of `networkx`

First, check which version of networkx you have installed. The cugraph plugin is only on the development version of NetworkX, `networkx-3.2rc0-dev0`. If the below command does not show this version, you will need to follow the instructions from `INSTALL.rst` on the NetworkX repository.

In [17]:
!pip show networkx

Name: networkx
Version: 3.2rc0.dev0
Summary: Python package for creating and manipulating graphs and networks
Home-page: 
Author: 
Author-email: Aric Hagberg <hagberg@lanl.gov>
License: 
Location: /opt/conda/envs/rapids/lib/python3.10/site-packages
Editable project location: /dylanc/week4/networkx
Requires: 
Required-by: cfn-lint, cugraph-nx, mapclassify, osmnx, python-louvain, scikit-image


## FOR REPRODUCIBILITY / DEV WORK / TEMP

The following cells will
- uninstall the current version of networkx 
- clone the networkx repository one directory below the current working directory
- upgrade networkx to be the current development version with `pip install`

You may need to restart the kernel to have the development version of networkX be visible.

In [5]:
!pip uninstall --yes networkx

In [12]:
# Skip if current networkx repository already exists
!(cd ..; \
git clone https://github.com/networkx/networkx.git)

Cloning into 'networkx'...
remote: Enumerating objects: 65975, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 65975 (delta 3), reused 4 (delta 2), pack-reused 65942
Receiving objects: 100% (65975/65975), 91.34 MiB | 17.01 MiB/s, done.
Resolving deltas: 100% (43970/43970), done.


In [6]:
!(cd ../networkx; \
pip install --upgrade --target=/opt/conda/envs/rapids/lib/python3.10/site-packages -e .[default])
# below: old pip install command,
# pip install --target=/opt/conda/envs/rapids/lib/python3.10/site-packages -e .[default])

Obtaining file:///dylanc/week4/networkx
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 22.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 18.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 33.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [10]:
!pip show networkx

Name: networkx
Version: 3.2rc0.dev0
Summary: Python package for creating and manipulating graphs and networks
Home-page: 
Author: 
Author-email: Aric Hagberg <hagberg@lanl.gov>
License: 
Location: /opt/conda/envs/rapids/lib/python3.10/site-packages
Editable project location: /dylanc/week4/networkx
Requires: 
Required-by: cfn-lint, mapclassify, osmnx, python-louvain, scikit-image


In [1]:
import networkx
networkx, networkx.__version__

(<module 'networkx' from '/dylanc/week4/networkx/networkx/__init__.py'>,
 '3.2rc0.dev0')

## Using networkx

In [1]:
import networkx as nx
import pandas as pd

In [2]:
nx.__version__, pd.__version__

('3.2rc0.dev0', '2.0.3')

The 'email-Eu-core' dataset, made up on anonymized email data from a large European institution, is a directed graph with 25571 edges and 1005 nodes. The following block reads this dataset and loads into a networkx.Graph object. We then measure the performance of the betweenness centrality algorithm with an approximation of k=50 node samples.

In [3]:
df = pd.read_csv("datasets/email-Eu-core.csv", header=None, names=['src', 'dst', 'wgt'], sep=' ')
Gnx = nx.from_pandas_edgelist(df, source='src', target='dst', create_using=nx.Graph())

In [4]:
%%timeit
bc = nx.betweenness_centrality(Gnx, k=50)

553 ms ± 7.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Customization

Depending on your graph needs, `networkx` supports the ability to choose your dispatcher. A dispatchable networkX algorithm will use the chosen dispatcher to convert networkx Graph to the backend equivalent, dispatch the work, then convert back to the required nx form. More information here: https://networkx.org/documentation/stable/reference/classes/index.html#backends

In [6]:
import cugraph_nx as cnx

In [7]:
nx.__version__, pd.__version__, cnx.__version__

('3.2rc0.dev0', '2.0.3', '23.10.00')

### Method 1: Specify which backend to use within function call

In [8]:
%%timeit
bc = nx.betweenness_centrality(Gnx, k=50, backend='cugraph')

141 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Method 2: Perform backend conversion on a separate line

In [9]:
%%timeit
Gcnx = cnx.from_networkx(Gnx)

2.68 ms ± 43.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
Gcnx = cnx.from_networkx(Gnx)

In [11]:
%%timeit
res = nx.betweenness_centrality(Gcnx, k=50)

139 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## What algorithms are supported?

|        Algorithm            |  Type         | Implemented |   Notes
| ----------------------------|---------------|-------------| ---------
| Betweeneess Centrality      | Centrality    |      X      | cuGraph 23.10 nightly         
| Edge Betweenness Centrality | Centrality    |      X      | cuGraph 23.10 nightly
| Louvain                     | Community     |             | cuGraph 23.10 nightly (expected)

## Future plans

We want to get your feedback! As this project develops, we want you to voice your opinions on which algorithms we should prioritize.